### Visualize Annotations and Predictions

In [ ]:
import json
from pathlib import Path
import os
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import numpy as np
from PIL import Image

In [ ]:
def plot_coco_annotated_image(json_path, image_dir, output_path,
                              draw='bbox',
                              line_width=2,
                              line_color='r',
                              output_size=(10, 8),
                              dpi=100):
    """
    Saves an image annotated with COCO-format labels (bounding boxes or polygons) to a file.

    Args:
        json_path (str): Path to the COCO-format JSON file (with annotations for one image).
        image_dir (str): Directory where the image is stored.
        output_path (str): Path to save the annotated image.
        draw (str): What to draw — 'bbox' for bounding boxes or 'segmentation' for polygons.
        line_width (int): Width of the annotation lines.
        line_color (str or tuple): Color of the annotation lines, e.g., 'blue' or (1, 0, 0) for red in RGB.
        output_size (tuple): Output image size in inches (width, height) for matplotlib.
        dpi (int): Dots per inch (resolution) of the saved image.
    """
    # Load JSON
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # Extract image info and annotations
    image_info = coco_data['images'][0]
    annotations = coco_data['annotations']
    image_path = os.path.join(image_dir, image_info['file_name'])

    # Open image
    image = Image.open(image_path)
    fig, ax = plt.subplots(figsize=output_size, dpi=dpi)
    ax.imshow(image)

    for ann in annotations:
        if draw == 'bbox':
            x, y, w, h = ann['bbox']
            rect = patches.Rectangle((x, y), w, h,
                                     linewidth=line_width,
                                     edgecolor=line_color,
                                     facecolor='none')
            ax.add_patch(rect)
        elif draw == 'segmentation' and ann['segmentation']:
            for seg in ann['segmentation']:
                poly = Polygon([(seg[i], seg[i+1]) for i in range(0, len(seg), 2)],
                               linewidth=line_width,
                               edgecolor=line_color,
                               facecolor='none')
                ax.add_patch(poly)

    ax.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

def plot_csv_annotated_image(csv_path, image_dir, output_path,
                              line_width=2,
                              line_color='r',
                              output_size=(10, 8),
                              dpi=100):
    labels = pd.read_csv(csv_path)

    if len(labels["image_path"].unique()) > 1:
        raise ValueError("Predictions contain multiple images.")

    image_path = os.path.join(image_dir, labels["image_path"].iloc[0])

    # Open image
    image = Image.open(image_path)
    fig, ax = plt.subplots(figsize=output_size, dpi=dpi)
    ax.imshow(image)

    for _, row in labels.iterrows():
        x_min = row["xmin"]
        x_max = row["xmax"]
        y_min = row["ymin"]
        y_max = row["ymax"]

        width = x_max - x_min
        height = y_max - y_min
        rect = patches.Rectangle((row["xmin"], row["ymin"]), width, height,
                                linewidth=line_width,
                                edgecolor=line_color,
                                facecolor='none')
        ax.add_patch(rect)

    ax.axis('off')
    if not isinstance(output_path, Path):
        output_path = Path(output_path)
    output_path.parent.mkdir(exist_ok=True, parents=True)

    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)


In [ ]:
line_color = tuple(np.array([255, 89, 0, 255]) / 255)
output_size = (8, 8)

image_dir = Path("G:/3D-GeoInfo-2025/data/1_base_data/images/")
kwargs = {"line_width": 3, "line_color": line_color, "output_size": output_size, "dpi": 80}


##### Create images for Fig. 1 - Example Annotations

In [ ]:
annotations_path = "C:/Users/cgs/Downloads/20230720_Sauen_3512a1_8901_115852_crown_polygons_coco.json"
plot_coco_annotated_image(annotations_path, image_dir, "manual_labeling_polygons.png", draw="segmentation", **kwargs)

image_dir = Path("G:/3D-GeoInfo-2025/data/1_base_data/images/")

annotations_path = image_dir.parent / "labels/manual_labeling/s1_p1_small_coco.json"
plot_coco_annotated_image(annotations_path, image_dir, "manual_labeling.png", **kwargs)

annotations_path = image_dir.parent / "labels/manual_correction/s1_p1_small_coco.json"
plot_coco_annotated_image(annotations_path, image_dir, "manual_correction.png", **kwargs)

annotations_path = image_dir.parent / "labels/automatic_labeling/s1_p1_small_coco.json"
plot_coco_annotated_image(annotations_path, image_dir, "automatic_labeling.png", **kwargs)

##### Create images for Fig. 5 and Fig. 7 - Example Predictions

In [ ]:
image_dir = Path("G:/3D-GeoInfo-2025/data/1_base_data/images/")

annotations_path = image_dir.parent / "labels/manual_labeling/s3_p1_coco.json"
plot_coco_annotated_image(annotations_path, image_dir, "manual_labeling_test_set.png", **kwargs)


In [ ]:
image_dir = "G:/3D-GeoInfo-2025/data/2_rescaled_data/2_5_cm/images"

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/baseline/test_predictions.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/baseline_2_5_cm.png", **kwargs)

image_dir = "G:/3D-GeoInfo-2025/data/2_rescaled_data/5_cm/images"

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/5_cm/baseline/test_predictions.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/baseline_5_cm.png", **kwargs)

image_dir = "G:/3D-GeoInfo-2025/data/2_rescaled_data/7_5_cm/images"

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/7_5_cm/baseline/test_predictions.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/baseline_7_5_cm.png", **kwargs)

image_dir = "G:/3D-GeoInfo-2025/data/2_rescaled_data/10_cm/images"

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/10_cm/baseline/test_predictions.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/baseline_10_cm.png", **kwargs)

image_dir = "G:/3D-GeoInfo-2025/data/2_rescaled_data/2_5_cm/images"

predictions_path = f"G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/manual_labeling_small/9_epochs/test_predictions_seed_2.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/manual_labeling_small.png", **kwargs)

predictions_path = f"G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/manual_labeling_ext/7_epochs/test_predictions_seed_0.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/manual_labeling_ext.png", **kwargs)

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/manual_correction_small/6_epochs/test_predictions_seed_0.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/manual_correction_small.png", **kwargs)

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/manual_correction_ext/9_epochs/test_predictions_seed_0.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/manual_correction_ext.png", **kwargs)

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/automatic_labeling_small/9_epochs/test_predictions_seed_4.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/automatic_labeling_small.png", **kwargs)

predictions_path = "G:/3D-GeoInfo-2025/data/4_predictions/2_5_cm/automatic_labeling_ext/8_epochs/test_predictions_seed_1.csv"
plot_csv_annotated_image(predictions_path, image_dir, "predictions/automatic_labeling_ext.png", **kwargs)
